In [8]:
from typing import Dict, List
from openai import AzureOpenAI
import sys
import math
import os
import re
import json
import pandas as pd

In [9]:
data = pd.read_csv('reviews.csv')
restarant = data[data['Restaurant'] == 'Beyond Flavours']
restarant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 60 to 69
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  10 non-null     object
 1   Reviewer    10 non-null     object
 2   Review      10 non-null     object
 3   Rating      10 non-null     int64 
 4   Metadata    10 non-null     object
dtypes: int64(1), object(4)
memory usage: 480.0+ bytes


In [10]:
reviews = restarant['Review'].tolist()
ratings = restarant['Rating'].tolist()
metadata = restarant['Metadata'].tolist()
print(len(reviews), len(ratings), len(metadata))

10 10 10


In [11]:
print(type(reviews[0]))
print(type(ratings[0]))
print(type(metadata[0]))

<class 'str'>
<class 'int'>
<class 'str'>


In [12]:
reviews

['we went for a team lunch, such a worst restaurant.. food was tasteless, service was very slow and Ac was not working. We’ve requested them multiple times but its of no use. Please don’t waste your money and I strictly recommend not to prefer beyond flavours,',
 'Food quality and taste is not good. Super slow service. Nice ambience but very poor in customer handling. Lot of items not available even if in the menu.',
 "Short review: Decent breads and starters, bad Shaam Savera, a little screwed up daal makhni (but should be a one-off case)\nBetter ambience at night... And importantly good quantity!\nI wanted to give a 2.5 but that's not an option 🙈\nExtensive...\nAmbience: It's better at night. Friends who had been here during the day didn't like it much. So if you prefer ambience, go at night!\nFood! Ordered a Hara Bhara Kebab, Shaam Savera, Daal Makhni, rotis and raita (Yes! I'm a vegetarian 🙈)\nThe good part! Quantity 😍\nI was excited to see Shaam Savera on their menu. But take it f

In [13]:
ratings

[1, 2, 3, 4, 5, 4, 1, 5, 1, 5]

In [14]:
metadata

['1 Review , 24 Followers',
 '3 Reviews , 1 Follower',
 '9 Reviews , 8 Followers',
 '18 Reviews , 10 Followers',
 '47 Reviews , 4734 Followers',
 '30 Reviews , 34 Followers',
 '1 Review , 1 Follower',
 '1 Review , 1 Follower',
 '13 Reviews , 67 Followers',
 '2 Reviews , 39 Followers']

In [15]:
user_input = "hello, can you tell me the rating of Beyond Flavours?"
prompt = """
        You are a restaurant recommendation system. Your task is to recognize the intention of the user based on their input.
        The user can ask for the rating of a restaurant, compare two restaurants, or ask for reviews of a restaurant.
        Your output should be a json string containing the intention and the arguments.
        The intention can be one of the following:
        1. rating: The user is asking for the rating of a restaurant.
        2. compare: The user is asking to compare two restaurants.
        3. review: The user is asking for reviews of a restaurant.
        The arguments should be a list of restaurant names. If the user is asking for the rating of a restaurant, the list should contain one restaurant name.
        If the user is asking to compare two restaurants, the list should contain two restaurant names.
        If the user is asking for reviews of a restaurant, the list should contain one restaurant name.
        # Example:
        ## User Input:
        What is the rating of Beyond Flavours?
        ## Output:
        {{"intention": "rating", "args": ["Beyond Flavours"]}}
        ## User Input:
        Beyond Flavours and Karachi Cafe, which one is better?
        ## Output:
        {{"intention": "compare", "args": ["Beyond Flavours", "Karachi Cafe"]}}
        ## User Input:
        What are people saying about Beyond Flavours?
        ## Output:
        {{"intention": "review", "args": ["Beyond Flavours"]}}

        The user input will be as follows:
        # User Input: 
        {user_input}
        """.format(user_input=user_input)

print(prompt)


        You are a restaurant recommendation system. Your task is to recognize the intention of the user based on their input.
        The user can ask for the rating of a restaurant, compare two restaurants, or ask for reviews of a restaurant.
        Your output should be a json string containing the intention and the arguments.
        The intention can be one of the following:
        1. rating: The user is asking for the rating of a restaurant.
        2. compare: The user is asking to compare two restaurants.
        3. review: The user is asking for reviews of a restaurant.
        The arguments should be a list of restaurant names. If the user is asking for the rating of a restaurant, the list should contain one restaurant name.
        If the user is asking to compare two restaurants, the list should contain two restaurant names.
        If the user is asking for reviews of a restaurant, the list should contain one restaurant name.
        # Example:
        ## User Input:
   

In [16]:
from main import fetch_restaurant_data
import pprint
result = fetch_restaurant_data("Squeeze @ The Lime")
pprint.pprint(result)

{'metadata': ['3 Reviews , 1 Follower',
              '2 Reviews , 1 Follower',
              '11 Reviews , 81 Followers',
              '155 Reviews , 1802 Followers',
              '3 Reviews , 4 Followers',
              '1 Review , 1 Follower',
              '6 Reviews , 30 Followers',
              '2 Reviews , 2 Followers',
              '7 Reviews , 1 Follower',
              '4 Reviews , 12 Followers'],
 'ratings': [1, 2, 3, 4, 5, 5, 1, 4, 3, 2],
 'restaurant': 'Squeeze @ The Lime',
 'reviews': ['Must read - I went to this place with my friends for buffet '
             'lunch. We had two options for welcome drink - one is pineapple '
             'and other is grape which were not natural fruit juices they '
             'tasted as preservatives, then moving on to soup - manchow soup '
             'was good, waiter forget to give us the starter. Served us after '
             'we reminding him, we liked crispy baby corn among 3 varieties of '
             'starters which he t

In [17]:
from main import IntentionRecognitionAgent

intention_recognition_agent = IntentionRecognitionAgent()
result = intention_recognition_agent.generate_response("What's the user rating for SKYHY?")
print(result)


{'intention': 'rating', 'args': ['SKYHY']}


In [18]:
print(type(result['args']))

<class 'list'>


In [19]:
from main import ReviewSummaryAgent
# fetch data 
data = fetch_restaurant_data("Squeeze @ The Lime")
ratings = data['ratings']
reviews = data['reviews']
metadata = data['metadata']

for r in reviews:
    print(r)
    print("===")

Must read - I went to this place with my friends for buffet lunch. We had two options for welcome drink - one is pineapple and other is grape which were not natural fruit juices they tasted as preservatives, then moving on to soup - manchow soup was good, waiter forget to give us the starter. Served us after we reminding him, we liked crispy baby corn among 3 varieties of starters which he took 10 mints to serve for the second serve when only 10% of the place was occupied. When I asked him mayo dip waiter was giving us the left over part of dip on the table beside. When I asked for chilled water they were saying they don't have, later one of my friend asked the same person he gave us chilled water. All the items which were placed at buffet were cold not even Luke warm. Biryani was totally dried out rice was sticking to the vessel had lot more bad experience here on same day
===
You need at least 2 hrs time to get your food prepare. better go for other restaurant instead of ordering foo

In [20]:
# agent
review_summary_agent = ReviewSummaryAgent()
result = review_summary_agent.generate_response(reviews)
print(result)

The restaurant offers a mix of experiences; while some enjoyed the excellent vegetarian food and memorable service during events, others faced issues with slow service, cold buffet items, and inconsistent food quality, particularly with sizzlers and certain dishes.


In [21]:
from main import FollowersParserAgent
from main import fetch_restaurant_data

# fetch data
data = fetch_restaurant_data("Squeeze @ The Lime")
ratings = data['ratings']
reviews = data['reviews']
metadata = data['metadata']

for m in metadata:
    print(m)
    print("===")

3 Reviews , 1 Follower
===
2 Reviews , 1 Follower
===
11 Reviews , 81 Followers
===
155 Reviews , 1802 Followers
===
3 Reviews , 4 Followers
===
1 Review , 1 Follower
===
6 Reviews , 30 Followers
===
2 Reviews , 2 Followers
===
7 Reviews , 1 Follower
===
4 Reviews , 12 Followers
===


In [22]:
# agent
followers_parser_agent = FollowersParserAgent()
result = followers_parser_agent.generate_response(metadata)

In [23]:
print(type(result[0]))
print(result)

<class 'int'>
[1, 1, 81, 1802, 4, 1, 30, 2, 1, 12]


In [24]:
from main import calculate_overall_rating
from main import fetch_restaurant_data
from main import FollowersParserAgent
import math
# fetch data
data = fetch_restaurant_data("SKYHY")
ratings = data['ratings']
reviews = data['reviews']
metadata = data['metadata']

print("ratings: ", ratings)
print("reviews: ", reviews)
print("metadata: ", len(metadata))

# parse followers
followers_parser_agent = FollowersParserAgent()
result = followers_parser_agent.generate_response(metadata)
print(result)

# follower = [2,61,23,295,9,2,14,55,21,1]
# weights = [math.log(i+1) for i in follower]
# print(weights)
# print(follower)



ratings:  [1, 2, 3, 4, 5, 3, 3, 4, 5, 1]
reviews:  ['Starters was ok to taste but after effect lasted for days. The colour of the stools was black for almost 5 days from the day we visited that place. My friends experience after having those starters was horrible.She was admitted in hospital.', "This place is okay at best. The drinks are too diluted with an extra crapload of ice. The live music was really great but didn't fit inside such a place. Cibo house is a much better place if you're looking for some good live music, a peaceful time and some great drinks with friends.", "I have been to this place with my friends on a Friday evening as they had Bollywood night starting from 8.\nThe food was average. They didn't even had Breezer which is disappointing. The service was not that great.\nAs mentioned earlier, They has Bollywood night which didn't even start till 10pm.\nFinally,\nFood - 2.5\nAmbience - 3\nService - 3\nMenu and pricing - 3", 'The biggest screen in town with a lovely roo

In [25]:
# calculate overall rating
overall_rating = calculate_overall_rating("SKYHY", ratings,result)
print(overall_rating)

3.457
